# SED for Trappist-1
This notebook executes the lines of code presented in Filippazzo et al. (submitted to PASP). Below I generate and analyze an SED for the M8 dwarfs Trappist-1 and Gl 752B. Then I create a catalog of SEDs for analysis.

In [1]:
# Imports
from astropy import units as q
from bokeh.io import output_notebook
from bokeh.plotting import show
from pkg_resources import resource_filename
output_notebook()

Loading BokehJS ...

## Initialization
Import the `SED` class and create a Trappist-1 object. The parallax, radius and spectral type are pulled from Simbad when available.

In [2]:
from sedkit import SED
trap1 = SED(name='Trappist-1')

[sedkit] SED initialized
[sedkit] Setting name to Trappist-1
[sedkit] 1 record for Trappist-1 found in Simbad.
[sedkit] Setting sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (23.10815789, -5.0413975)>
[sedkit] There was a problem determining the interstellar reddening. Setting to 0. You can manually set this with the 'reddening' attribute.
[sedkit] Setting parallax to (<Quantity 80.4512 mas>, <Quantity 0.1211 mas>) with reference '2018yCat.1345....0G'
[sedkit] Setting distance to (<Quantity 12.43 pc>, <Quantity 0.02 pc>, <Quantity 0. pc>) with reference '2018yCat.1345....0G'
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'
[sedkit] Setting radius to (<Quantity 0.15 solRad>, <Quantity 0.02 solRad>, <Quantity 0.01 solRad>) with reference '2013ApJS..208....9P'
[sedkit] Setting spectral_type to (67.5, 0.5, 'V', None, None) with reference '2000AJ....120.1085G'


## Photometry
Add photometry manually...

In [3]:
trap1.add_photometry('Johnson.V', 18.798, 0.082, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.R', 16.466, 0.065, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.I', 14.024, 0.115, ref='Costa et al. (2006)')

[sedkit] Setting Johnson.V photometry to 18.798 (0.082) with reference 'Costa et al. (2006)'
[sedkit] Setting Cousins.R photometry to 16.466 (0.065) with reference 'Costa et al. (2006)'
[sedkit] Setting Cousins.I photometry to 14.024 (0.115) with reference 'Costa et al. (2006)'


...and retrieve some from Simbad.

In [4]:
trap1.find_2MASS()
trap1.find_WISE()

[sedkit] 1 record found in II/246/out.
[sedkit] Setting 2MASS.J photometry to 11.354 (0.022) with reference 'II/246/out'
[sedkit] Setting 2MASS.H photometry to 10.718 (0.021) with reference 'II/246/out'
[sedkit] Setting 2MASS.Ks photometry to 10.296 (0.023) with reference 'II/246/out'
[sedkit] 1 record found in II/328/allwise.
[sedkit] Setting WISE.W1 photometry to 10.068 (0.023) with reference 'II/328/allwise'
[sedkit] Setting WISE.W2 photometry to 9.807 (0.020) with reference 'II/328/allwise'
[sedkit] Setting WISE.W3 photometry to 9.510 (0.044) with reference 'II/328/allwise'
[sedkit] Setting WISE.W4 photometry to 8.565 (nan) with reference 'II/328/allwise'


In [5]:
# Print the photometry table
trap1.photometry[['band', 'eff', 'app_magnitude', 'app_magnitude_unc', 'ref']].pprint()

   band        eff      app_magnitude app_magnitude_unc         ref        
                um                                                         
--------- ------------- ------------- ----------------- -------------------
Johnson.V 0.54638671875          18.8             0.082 Costa et al. (2006)
Cousins.R  0.6357421875         16.47             0.065 Costa et al. (2006)
Cousins.I 0.78271484375         14.02             0.115 Costa et al. (2006)
  2MASS.J  1.2353515625         11.35             0.022          II/246/out
  2MASS.H   1.662109375         10.72             0.021          II/246/out
 2MASS.Ks   2.158203125          10.3             0.023          II/246/out
  WISE.W1   3.353515625         10.07             0.023      II/328/allwise
  WISE.W2     4.6015625         9.805              0.02      II/328/allwise
  WISE.W3       11.5625          9.51             0.044      II/328/allwise
  WISE.W4      22.09375          8.56               nan      II/328/allwise


## Spectra

In [6]:
# Grab spectrum files
nir_spec = resource_filename('sedkit', 'data/Trappist-1_NIR.fits')
opt_spec = resource_filename('sedkit', 'data/Trappist-1_OPT.fits')

# Add the spectra (trimming edge effects off the optical one)
trap1.add_spectrum_file(nir_spec, name='NIR spectrum', wave_units=q.um, flux_units=q.W/q.m**2/q.um, ref='Filippazzo et al. (2015)')
trap1.add_spectrum_file(opt_spec, name='OPT spectrum', wave_units=q.AA, flux_units=q.erg/q.s/q.cm**2/q.AA, trim={'exclude': [(0.9*q.um, 2*q.um)]}, ref='Cruz et al. (2003)')

# Print the table
trap1.spectra[['name', 'wave_min', 'wave_max', 'resolution', 'ref']].pprint()

[sedkit] Spectrum added.
[sedkit] Spectrum added.
    name        wave_min      wave_max   resolution           ref           
                   um            um                                         
------------ ------------- ------------- ---------- ------------------------
NIR spectrum  0.6533203125    2.56640625        563 Filippazzo et al. (2015)
OPT spectrum 0.60009765625 0.89990234375       1033       Cruz et al. (2003)


## Other Data
Set the spectral type.

In [7]:
trap1.spectral_type = 'M8', '2006AJ....132.1234C'

[sedkit] Setting spectral_type to (68.0, 0.5, 'V', None, None) with reference '2006AJ....132.1234C'


Set the age.

In [8]:
trap1.age = 7.6*q.Gyr, 2.2*q.Gyr, '2017ApJ...845..110B'

[sedkit] Setting age to (<Quantity 7.6 Gyr>, <Quantity 2.2 Gyr>) with reference '2017ApJ...845..110B'


Set the age.

In [9]:
trap1.radius = 0.121*q.R_sun, 0.003*q.R_sun, '2021PSJ.....2....1A'

[sedkit] Setting radius to (<Quantity 0.121 solRad>, <Quantity 0.003 solRad>) with reference '2021PSJ.....2....1A'


## Build SED

In [10]:
trap1.make_sed()

[sedkit] Setting fbol to (<Quantity 2.21148156e-10 erg / (cm2 s)>, <Quantity 5.96708612e-12 erg / (cm2 s)>) with reference 'This Work'
[sedkit] Setting Lbol to (<Quantity 4.08824142e+30 erg / s>, <Quantity 1.11091913e+29 erg / s>) with reference 'This Work'
[sedkit] Setting Lbol_sun to (-2.971, 0.012) with reference 'This Work'
[sedkit] Setting mbol to (12.656, 0.029) with reference 'This Work'
[sedkit] Setting Mbol to (12.184, 0.029) with reference 'This Work'
[sedkit] Setting logg to [2.0, 0.06] with reference 'parsec12_solar'
[sedkit] Setting mass to [<Quantity 0.551 solMass>, <Quantity 0.02 solMass>] with reference 'parsec12_solar'
[sedkit] Setting Teff to (<Quantity 2999 K>, <Quantity 42 K>) with reference 'This Work'


## Fit the data

In [11]:
# Fit to infer spectral type
trap1.fit_spectral_type()

[sedkit] Best fit SpeX Prism Library spec: Opt:M8V
[sedkit] Setting Lbol to (<Quantity 4.08824142e+30 erg / s>, <Quantity 1.11091913e+29 erg / s>) with reference 'This Work'
[sedkit] Setting Lbol_sun to (-2.971, 0.012) with reference 'This Work'
[sedkit] Setting Mbol to (12.184, 0.029) with reference 'This Work'
[sedkit] Setting Teff to (<Quantity 2999 K>, <Quantity 42 K>) with reference 'This Work'


In [12]:
# Fit to infer fundamental params
from sedkit import BTSettl
trap1.fit_modelgrid(BTSettl())

[sedkit] Best fit BT-Settl spec: 2500.0/5.5/0.0/0.0
[sedkit] Setting Lbol to (<Quantity 4.08824142e+30 erg / s>, <Quantity 1.11091913e+29 erg / s>) with reference 'This Work'
[sedkit] Setting Lbol_sun to (-2.971, 0.012) with reference 'This Work'
[sedkit] Setting Mbol to (12.184, 0.029) with reference 'This Work'
[sedkit] Setting Teff to (<Quantity 2999 K>, <Quantity 42 K>) with reference 'This Work'


## Inferring mass, surface gravity, radius, and Teff
Set the evolutionary model grid and optionally infer fundamental parameters.

In [13]:
trap1.evo_model = 'DUSTY00'

In [14]:
trap1.infer_logg(plot=True)

[sedkit.Isochrone] -2.983: Lbol must be between -5.35 and -3.08 to infer logg from DUSTY00 isochrones.
[sedkit.Isochrone] -2.971: Lbol must be between -5.35 and -3.08 to infer logg from DUSTY00 isochrones.
[sedkit.Isochrone] -2.959: Lbol must be between -5.35 and -3.08 to infer logg from DUSTY00 isochrones.
[sedkit] Could not calculate surface gravity.


In [15]:
trap1.infer_mass(plot=True, mass_units=q.Mjup)

[sedkit.Isochrone] -2.983: Lbol must be between -5.35 and -3.08 to infer mass from DUSTY00 isochrones.
[sedkit.Isochrone] -2.971: Lbol must be between -5.35 and -3.08 to infer mass from DUSTY00 isochrones.
[sedkit.Isochrone] -2.959: Lbol must be between -5.35 and -3.08 to infer mass from DUSTY00 isochrones.


## Results
The results of the SED calculations can be accessed anytime with the `results` property.

In [16]:
trap1.results

param,value,unc,units
object,object,object,object
name,Trappist-1,--,--
ra,23.10815788888889,--,--
dec,-5.0413974999999995,--,--
age,7.6,2.2,Gyr
membership,--,--,--
distance,12.43,0.02,pc
parallax,80.4512,0.1211,mas
SpT,M8V,--,--
spectral_type,68.0,0.5,--


The `plot` method plots the SED, spectra, and photometry by default. The best fits and integral surface used to calculate `Lbol` can be included by setting their arguments to `True`. Click on the elements in the legend to toggle.

In [17]:
plt = trap1.plot(integral=True, best_fit=True)

Check out the references of all the data used in the SED with the `refs` property.

In [18]:
trap1.refs

{'parallax': '2018yCat.1345....0G',
 'distance': '2018yCat.1345....0G',
 'reddening': '2018JOSS....3..695M',
 'radius': '2021PSJ.....2....1A',
 'spectral_type': '2006AJ....132.1234C',
 'age': '2017ApJ...845..110B',
 'fbol': 'This Work',
 'Lbol': 'This Work',
 'Lbol_sun': 'This Work',
 'mbol': 'This Work',
 'Mbol': 'This Work',
 'Teff': 'This Work',
 'photometry': {'Johnson.V': 'Costa et al. (2006)',
  'Cousins.R': 'Costa et al. (2006)',
  'Cousins.I': 'Costa et al. (2006)',
  '2MASS.J': 'II/246/out',
  '2MASS.H': 'II/246/out',
  '2MASS.Ks': 'II/246/out',
  'WISE.W1': 'II/328/allwise',
  'WISE.W2': 'II/328/allwise',
  'WISE.W3': 'II/328/allwise',
  'WISE.W4': 'II/328/allwise'},
 'spectra': {'NIR spectrum': 'Filippazzo et al. (2015)',
  'OPT spectrum': 'Cruz et al. (2003)'},
 'best_fit': {'SpeX Prism Library spec': '2014ASInC..11....7B',
  'BT-Settl spec': '2014IAUS..299..271A'}}

And look at all the best fit models with the `best_fit` property.

In [19]:
trap1.best_fit

{'SpeX Prism Library spec': {'spty': 'Opt:M8V',
  'filepath': '/Users/jfilippazzo/Documents/Modules/sedkit/sedkit/data/models/atmospheric/spexprismlibrary/spex-prism_VB10_20030919_BUR04B.txt.xml',
  'spectrum': array([[6.5798396e-01, 6.6016799e-01, 6.6234499e-01, ..., 2.5545900e+00,
          2.5575399e+00, 2.5605099e+00],
         [4.1532196e-16, 3.3675452e-16, 3.1890738e-16, ..., 1.4468625e-15,
          1.6911287e-15, 1.4947845e-15]], dtype=float32),
  'label': 'Opt:M8V',
  'SpT': 68.0,
  'gstat': 0.01746099602179178,
  'const': 9.907370754469251e-14,
  'full_model': <sedkit.spectrum.Spectrum at 0x1a6b772b0>,
  'wave_units': Unit("um"),
  'flux_units': Unit("erg / (Angstrom cm2 s)"),
  'fit_to': 'spec'},
 'BT-Settl spec': {'alpha': 0.0,
  'filepath': '/Users/jfilippazzo/Documents/Modules/sedkit/sedkit/data/models/atmospheric/btsettl/lte025-5.5-0.0.BT-Settl.7.dat.xml',
  'label': '2500.0/5.5/0.0/0.0',
  'logg': 5.5,
  'meta': 0.0,
  'spectrum': array([[3.00001489e-01, 3.02471652e-01,